In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
# Import of keras model and hidden layers for our convolutional network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Flatten, Input, BatchNormalization, Dropout
from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)

Model trained in two categories fist, and palm. 
Trained on samples cutted from 


In [ ]:
batch_size = 16
img_size = (128,128)
NUM_CLASSES = 2

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=45,
    width_shift_range=[-20,20],
    height_shift_range=[-20,20],
    brightness_range=None,
    shear_range=0.0,
    zoom_range=[0.8,1.2],
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=True,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None
)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=90,
    width_shift_range=[-20,20],
    height_shift_range=[-20,20],
    brightness_range=[0.8,1.2],
    shear_range=0.0,
    zoom_range=[0.8,1.2],
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()


train_generator = train_datagen.flow_from_directory(
    directory='../../DATA/data/hands/train/images',
    shuffle=True,
    target_size=img_size,
    color_mode='rgb',
    batch_size=batch_size)
print('Train generator created\n')

valid_generator = valid_datagen.flow_from_directory(
    directory='../../DATA/data/hands/valid/images',
    shuffle=True,
    target_size=img_size,
    color_mode='rgb',
    batch_size=batch_size)
print('Valid generator created\n')


test_generator = test_datagen.flow_from_directory(
    directory = '../../DATA/data/hands/test/images',
    shuffle = True,
    target_size = img_size,
    color_mode='rgb',
    batch_size=batch_size

)
print('Train generator created\n')

In [ ]:
print(next(train_generator)[0].shape)

In [ ]:
model.evaluate(test_generator)

In [ ]:
model.save('trained_palm_fist.h5')


In [ ]:
from tensorflow.keras.applications import EfficientNetB0


def build_model(num_classes):
    model = EfficientNetB0(include_top=False,input_shape=(128,128,3), weights="imagenet")
    inputs = model.input
    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam()
    loss = tf.keras.losses.CategoricalCrossentropy()

    model.compile(
        optimizer=optimizer, loss=loss, metrics=["accuracy"]
    )
    return model

In [ ]:

model = build_model(num_classes=NUM_CLASSES)


In [ ]:
model.summary()

In [ ]:
import matplotlib.pyplot as plt


def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


In [ ]:
steps_per_epoch = train_generator.n // batch_size
validation_steps = valid_generator.n // batch_size


epochs = 10 
hist = model.fit(train_generator,epochs=epochs,validation_data=valid_generator, verbose=2)
plot_hist(hist)

In [ ]:
def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    loss = tf.keras.losses.CategoricalCrossentropy()

    model.compile(
        optimizer=optimizer, loss=loss, metrics=["accuracy"]
    )


unfreeze_model(model)

epochs = 10 
hist = model.fit(train_generator,epochs=epochs,validation_data=valid_generator, verbose=2)

In [ ]:
model.save('trained_EfficientNetB0_palm_fist.h5')


In [ ]:
model.evaluate(test_generator)